In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import ase.io

from aml_storage import Labels, Block, Descriptor

from utils.rascaline import RascalineSphericalExpansion
from utils.models.soap import compute_power_spectrum
import utils.io

In [3]:
frames = ase.io.read("data/molecule_conformers_dftb.xyz", ":100")

# frames = ase.io.read("selection-10k.extxyz", "::100")
for frame in frames:
    frame.cell = [0, 0, 0]
    frame.pbc = [False, False, False]

RASCALINE_HYPERS = {
    "cutoff": 3.5,
    "atomic_gaussian_width": 0.3,
    "max_radial": 6,
    "max_angular": 6,
    "radial_basis": {"Gto": {}},
    "cutoff_function": {"ShiftedCosine": {"width": 0.5}},
    "gradients": True,
}

calculator = RascalineSphericalExpansion(RASCALINE_HYPERS)
spx = calculator.compute(frames)

In [4]:
utils.io.write("spx.h5", spx)
spx_loaded = utils.io.read("spx.h5")

In [7]:
assert np.all(spx.sparse == spx_loaded.sparse)
assert np.all(spx.block(0).values == spx_loaded.block(0).values)
assert np.all(spx.block(0).gradient("positions").data == spx_loaded.block(0).gradient("positions").data)
assert np.all(spx.block(0).gradient("positions").samples == spx_loaded.block(0).gradient("positions").samples)

for i, expected in enumerate(spx.block(0).gradient("positions").components):
    assert np.all(expected == spx_loaded.block(0).gradient("positions").components[i])

In [8]:
soap = compute_power_spectrum(spx)

In [9]:
utils.io.write("soap.h5", soap)
soap = utils.io.read("soap.h5")